In [1]:
import $ivy.`org.apache.spark::spark-sql:3.2.0` // Or use any other 2.x version here
import $ivy.`sh.almond::almond-spark:0.10.1`

import org.apache.spark._
import org.apache.spark.sql.{functions => func, _}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

import org.slf4j.LoggerFactory
import org.apache.log4j.{Level, Logger}

val spark = SparkSession
      .builder()
      .config("spark.sql.shuffle.partitions", 4)
      .master("local[4]")
      .getOrCreate()

import spark.implicits._

Logger.getRootLogger().setLevel(Level.ERROR)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
22/06/02 20:43:02 INFO SparkContext: Running Spark version 3.2.0
22/06/02 20:43:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/06/02 20:43:02 INFO ResourceUtils: ==============================================================
22/06/02 20:43:02 INFO ResourceUtils: No custom resources configured for spark.driver.
22/06/02 20:43:02 INFO ResourceUtils: ==============================================================
22/06/02 20:43:02 INFO SparkContext: Submitted application: beb3f7e7-7040-4f94-a6d3-f22bfcb66f91
22/06/02 20:43:02 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, amount: 1.0)
22/06/0

import $ivy.$                                   // Or use any other 2.x version here

import $ivy.$                               


import org.apache.spark._

import org.apache.spark.sql.{functions => func, _}

import org.apache.spark.sql.functions._

import org.apache.spark.sql.types._


import org.slf4j.LoggerFactory

import org.apache.log4j.{Level, Logger}


spark: SparkSession = org.apache.spark.sql.SparkSession@45bab224
import spark.implicits._



In [4]:
import org.apache.spark.sql.expressions.Window

import org.apache.spark.sql.expressions.Window

In [5]:
val lapTimeToMs = (time: String) => {
    val regex = """([0-9]|[0-9][0-9]):([0-9][0-9])\.([0-9][0-9][0-9])""".r
    time match {
        case regex(min,sec,ms) => min.toInt * 60 * 1000 + sec.toInt * 1000 + ms.toInt
        case "\\N" => 180000
    }
    
}: Long

lapTimeToMs: String => Long = ammonite.$sess.cmd4$Helper$$Lambda$3730/1713426558@4864a4b9

In [6]:
val lapTimeToMsUDF = udf(lapTimeToMs)
spark.udf.register("lapTimeToMs", lapTimeToMsUDF)

lapTimeToMsUDF: expressions.UserDefinedFunction = SparkUserDefinedFunction(
  ammonite.$sess.cmd4$Helper$$Lambda$3730/1713426558@4864a4b9,
  LongType,
  List(
    Some(
      ExpressionEncoder(
        StaticInvoke(
          class org.apache.spark.unsafe.types.UTF8String,
          StringType,
          "fromString",
          List(BoundReference(0, ObjectType(class java.lang.String), true)),
          List(),
          true,
          false
        ),
        Invoke(
          UpCast(
            GetColumnByOrdinal(0, StringType),
            StringType,
            List("- root class: \"java.lang.String\"")
          ),
          "toString",
          ObjectType(class java.lang.String),
          List(),
          List(),
          true,
          false
        ),
        java.lang.String
      )
    )
  ),
  Some(
    ExpressionEncoder(
      BoundReference(0, LongType, false),
      AssertNotNull(
        UpCast(
          GetColumnByOrdinal(0, LongType),
          LongType,
...
r

In [7]:
val msToLapTime = (time: Long) => {
    val mins = time / 60000
    val secs = (time - mins*60000)/1000
    val ms = time - mins*60000 - secs*1000
    
    val formattedSecs = if((secs / 10).toInt == 0) "0" + secs else secs
    // if ms = 00x -> "0"+"0"+x . if ms = 0xx -> "0"+ms
    val formattedMs = if((ms / 100).toInt == 0) "0" + (if((ms / 10).toInt == 0) "0" + ms else ms) else ms
    mins + ":" + formattedSecs + "." + formattedMs    
}: String

msToLapTime: Long => String = ammonite.$sess.cmd6$Helper$$Lambda$3978/25967365@7cd86df3

In [8]:
msToLapTime(lapTimeToMs("1:30.234"))

res7: String = "1:30.234"

In [9]:
val msToLapTimeUDF = udf(msToLapTime)
spark.udf.register("msToLapTime", msToLapTimeUDF)

msToLapTimeUDF: expressions.UserDefinedFunction = SparkUserDefinedFunction(
  ammonite.$sess.cmd6$Helper$$Lambda$3978/25967365@7cd86df3,
  StringType,
  List(
    Some(
      ExpressionEncoder(
        BoundReference(0, LongType, false),
        AssertNotNull(
          UpCast(
            GetColumnByOrdinal(0, LongType),
            LongType,
            List("- root class: \"scala.Long\"")
          ),
          List("- root class: \"scala.Long\"")
        ),
        Long
      )
    )
  ),
  Some(
    ExpressionEncoder(
      StaticInvoke(
        class org.apache.spark.unsafe.types.UTF8String,
        StringType,
        "fromString",
        List(BoundReference(0, ObjectType(class java.lang.String), true)),
        List(),
        true,
        false
      ),
      Invoke(
        UpCast(
          GetColumnByOrdinal(0, StringType),
          StringType,
          List("- root class: \"java.lang.String\"")
        ),
        "toString",
        ObjectType(class java.lang.String),


In [16]:
val races = spark.read.format("csv")
    .option("header", "true")
    .option("sep", ",")
    .load("../data/races.csv")

val lap_times = spark.read.format("csv")
    .option("header", "true")
    .option("sep", ",")
    .load("../data/lap_times.csv")

val drivers = spark.read.format("csv")
    .option("header", "true")
    .option("sep", ",")
    .load("../data/drivers.csv")

val results = spark.read.format("csv")
    .option("header", "true")
    .option("sep", ",")
    .load("../data/results.csv")
    .where(col("fastestLapTime") =!= "192.074")

races: DataFrame = [raceId: string, year: string ... 6 more fields]
lap_times: DataFrame = [raceId: string, driverId: string ... 4 more fields]
drivers: DataFrame = [driverId: string, driverRef: string ... 7 more fields]
results: Dataset[Row] = [resultId: string, raceId: string ... 16 more fields]

In [15]:
results.show()

+--------+------+--------+-------------+------+----+--------+------------+-------------+------+----+---------+------------+----------+----+--------------+---------------+--------+
|resultId|raceId|driverId|constructorId|number|grid|position|positionText|positionOrder|points|laps|     time|milliseconds|fastestLap|rank|fastestLapTime|fastestLapSpeed|statusId|
+--------+------+--------+-------------+------+----+--------+------------+-------------+------+----+---------+------------+----------+----+--------------+---------------+--------+
|   25174|  1062|     830|            9|    33|   3|       9|           9|            9|     2|  70|+1:20.244|     7563443|        43|   4|       192.074|        194.843|       1|
+--------+------+--------+-------------+------+----+--------+------------+-------------+------+----+---------+------------+----------+----+--------------+---------------+--------+



In [21]:
def queryConsistencia(races:DataFrame, lap_times:DataFrame, drivers:DataFrame, results:DataFrame):DataFrame = {
    val driverWindow = Window.partitionBy("driverId")
    val seasonWindow = Window.partitionBy("year")

    
    val races_filtered = races
        .where(col("year") === 2012)
    
    val avg_lap_times = lap_times
        .withColumnRenamed("time", "lapTime")
        .join(races_filtered, Seq("raceId"), "right")
        .withColumn("milliseconds", col("milliseconds").cast(IntegerType))
        .withColumn("avgMs", avg(col("milliseconds")).over(driverWindow))
        .dropDuplicates("driverId")
        .select("driverId", "avgMs")
    
    val lapCount = lap_times
        .join(races_filtered, Seq("raceId"), "right")
        .withColumn("lapsPerDriver", count(col("lap")).over(driverWindow))

        val (distinctDrivers, allLaps) = lapCount
            .agg(
                countDistinct("driverID"),
                count(col("lap"))
        ).as[(BigInt, BigInt)]
        .collect()(0)

    val avgLapsThisPeriod = allLaps.toInt / distinctDrivers.toInt


    val experiencedDrivers = lapCount
        .where(col("lapsPerDriver") >= avgLapsThisPeriod)
        .select("driverId")
        .distinct()
        .as[String]
        .collect()
    
    results.join(races, Seq("raceId"), "right")
        .na.drop(Seq("fastestLapTime"))
        .withColumn("fastestLapTimeMs", lapTimeToMsUDF(col("fastestLapTime")))
        .withColumn("avgFastestLapMs", avg(col("fastestLapTimeMs")).over(driverWindow))
        .dropDuplicates("driverId")
        .join(avg_lap_times, Seq("driverId"), "left")
        .withColumn("diffLapTimes", abs('avgMs - 'avgFastestLapMs).cast(IntegerType))
        .withColumn("avgDiff", msToLapTimeUDF(col("diffLapTimes").cast(IntegerType)))
        .where(col("driverId").isInCollection(experiencedDrivers))
        .join(drivers, "driverId")
        .withColumn("driver", concat(col("forename"), lit(" "), col("surname")))
        .select("driver", "avgDiff")
        .orderBy("avgDiff")
}

defined function queryConsistencia

In [22]:
queryConsistencia(races,lap_times,drivers,results).show(30,false)

+--------+------+--------+-------------+------+----+--------+------------+-------------+------+----+-----------+------------+----------+----+--------------+---------------+--------+----+-----+---------+---------------------+----------+--------+-------------------------------------------------------+----------------+------------------+------------------+------------+--------+-----------+------+----+---------+----------+----------+-----------+--------------------------------------------------------+-----------------+
|driverId|raceId|resultId|constructorId|number|grid|position|positionText|positionOrder|points|laps|time       |milliseconds|fastestLap|rank|fastestLapTime|fastestLapSpeed|statusId|year|round|circuitId|name                 |date      |time    |url                                                    |fastestLapTimeMs|avgFastestLapMs   |avgMs             |diffLapTimes|avgDiff |driverRef  |number|code|forename |surname   |dob       |nationality|url                               

|808     |337   |20343   |4            |12    |17  |\N      |R           |21           |0     |13  |\N         |\N          |7         |19  |2:04.307      |182.422        |22      |2010|1    |3        |Bahrain Grand Prix   |2010-03-14|12:00:00|http://en.wikipedia.org/wiki/2010_Bahrain_Grand_Prix   |124307          |99824.3275862069  |104645.06370656371|4820        |0:04.820|petrov     |\N    |PET |Vitaly   |Petrov    |1984-09-08|Russian    |http://en.wikipedia.org/wiki/Vitaly_Petrov              |Vitaly Petrov    |
|811     |337   |20341   |164          |21    |23  |\N      |R           |19           |0     |17  |\N         |\N          |14        |22  |2:09.127      |175.613        |25      |2010|1    |3        |Bahrain Grand Prix   |2010-03-14|12:00:00|http://en.wikipedia.org/wiki/2010_Bahrain_Grand_Prix   |129127          |103025.67391304347|103420.51043643264|394         |0:00.394|bruno_senna|\N    |SEN |Bruno    |Senna     |1983-10-15|Brazilian  |http://en.wikipedia.org/wiki/Bruno

In [12]:
// mostConsistentDriver.collect

In [13]:
def run[A](code: => A): Long = {
    val start = System.currentTimeMillis()
    val res = code
    System.currentTimeMillis() - start
}

def average(list: List[Long]):Long = list.sum / list.size.toLong

def timeTest(f: => Long): Long = {
    val list = (1 to 1).map(_ => f).toList
    average(list)
}

val res = timeTest(run(mostConsistentDriver.collect))

defined function run
defined function average
defined function timeTest
res: Long = 1032L